In [ ]:
%cd ~/contracode
import numpy as np
import pickle
import gzip
from tqdm.auto import tqdm
import pandas as pd
import time
from typing import Iterable
from loguru import logger
import multiprocessing as mp
# import modin.pandas as pd
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from transformers import BertTokenizerFast
from representjs import DATA_DIR
from tqdm import tqdm
# import swifter

In [10]:
train_df

,data_idx,text
0,1319387,const x = function (t) {\n return t.preventDe...
1,1290376,"function x(schema, options) {\n var discover;..."
2,1759800,"const x = function (context, node, args, conta..."
3,1829150,"const x = function (c, idx) {\n var cachedVer..."
4,293100,const x = function() {\n var r;\n return...
...,...,...
18767786,1220661,function x() {\n var _params = this...
18767787,1548900,"const x = function (e, t) {\n var i;\n retur..."
18767788,239958,"function x(_, handler) {\n if (handlers.len..."
18767789,1694742,const x = function () {\n function t(t) {\n ...


In [ ]:
# test_df = load_data("/data/ajay/contracode/data/hf_data/augmented_pretrain_df.test.pickle.gz")
# print("loaded test df")
# test_df['toks'] = test_df['text'].apply(lambda x: np.asarray(tokenizer.encode(x)))
# test_df_out = test_df[['toks', 'data_idx']].copy()

In [ ]:
# test_df_out.to_pickle("/home/ajay/contracode/data/hf_data/augmented_pretrain_df_tok.test.pickle.gz")

In [ ]:
# train_df = load_data("/home/ajay/contracode/data/hf_data/augmented_pretrain_df.train.pickle.gz")
# print("Loaded train_df")

In [ ]:
# train_df['toks'] = train_df['text'].apply(lambda x: np.asarray(tokenizer.encode(x)))
# print("applied")

In [ ]:
# train_df_out = train_df[['toks', 'data_idx']].copy()
# print("dropped column")

In [ ]:
# train_df.to_parquet("/data/ajay/augmented_pretrain_df_tok.train.parquet")
# print("saved")

# Shard the train set for tokenization

In [ ]:
# train_df = load_data("/data/ajay/contracode/data/hf_data/augmented_pretrain_df.train.pickle.gz")

In [33]:
from multiprocessing.pool import ThreadPool

def split_df(df, save_pattern, num_chunks=160):
    # Split data frame into chunks
    chunk_size = int(df.shape[0] / num_chunks)
    def save_data(data):
        chunk_i, start, chunk_size, df, save_pattern = data
        save_path = save_pattern.format(chunk_i)
        df_subset = df.iloc[start : start + chunk_size]
        df_subset.to_pickle(save_path)
        print("Saved ", save_path)
    items = [(i, start, chunk_size, df, save_pattern) for i, start in enumerate(range(0, df.shape[0], chunk_size))]
    with ThreadPool(64) as pool:
        pool.map(save_data, items)

In [34]:
chunk_dir = "/data/ajay/contracode/data/hf_data/train_chunks"
!mkdir -p {chunk_dir}
split_df(train_df, chunk_dir + "/augmented_pretrain_df.{:04d}.train.pickle.gz", 160)

Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0002.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0009.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0012.train.pickle.gz
Saved Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0006.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0022.train.pickle.gz
 /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0000.train.pickle.gzSaved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0019.train.pickle.gz

Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0004.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0026.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0016.train.pickle.gz
Saved  /data/ajay/co

Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0040.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0033.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0048.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0045.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0023.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0063.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0024.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0055.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0035.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0012.train.pickle.gz
Saved  /data/ajay/co

Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0101.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0105.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0106.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0100.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0102.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0118.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0122.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0110.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0109.train.pickle.gz
Saved  /data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0123.train.pickle.gz
Saved  /data/ajay/co

In [43]:
tqdm.pandas()

path = "/data/ajay/contracode/data/hf_data/train_chunks/augmented_pretrain_df.0000.train.pickle.gz"

def load_tokenizer(path="data/vocab/8k_bpe/8k_bpe-vocab.txt"):
    return BertTokenizerFast(path, clean_text=True, lowercase=False, strip_accents=True, unk_token="<unk>")

def load_data(path):
    return pd.read_pickle(path)

tokenizer = load_tokenizer()
df_shard = load_data(path)
df_shard['toks'] = df_shard['text'].progress_apply(lambda x: np.asarray(tokenizer.encode(x)))
df_shard = df_shard[['data_idx', 'toks']]

/home/ajay/miniconda3/envs/paras/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 117298/117298 [02:27<00:00, 793.01it/s] 


In [52]:
from tqdm.contrib.concurrent import process_map

dfs = []

files = []
for i in tqdm(list(range(161))):
    path = f"/data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.{i:04d}.train.pickle.gz"
    files.append(path)

def load_file(fname):
    out = pd.read_pickle(fname)
    return out
    
dfs = process_map(load_file, files, max_workers=16)

100%|██████████| 161/161 [00:00<00:00, 6202.48it/s]


DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0000.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0009.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0003.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0005.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0008.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0006.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0013.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_to

DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0063.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0064.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0065.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0066.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0067.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0068.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0069.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_to

DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0122.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0025.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0123.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0124.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0022.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0125.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0126.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_to

 /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0158.train.pickle.gz
DONE Loaded file DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0023.train.pickle.gz
 /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0160.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0159.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0008.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0010.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0044.train.pickle.gz
DONE Loaded file  /data/ajay/contracode/data/hf_data/train_chunks_tokenized/augmented_pretrain_tokenized_df.0005.t

In [58]:
merged_df = pd.concat(dfs)

In [61]:
merged_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18767791 entries, 0 to 18767790
Data columns (total 2 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   data_idx  int64 
 1   toks      object
dtypes: int64(1), object(1)
memory usage: 7.0 GB


In [ ]:
merged_df.to_pickle('/data/ajay/contracode/data/hf_data/merged_tok.pickle.gz')